In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from xgboost import XGBClassifier

df = pd.read_csv("/content/dataset.csv")

feature_cols = [
    'PTS_x', 'REB_x', 'AST_x', 'STL_x', 'BLK_x', 'MIN_x',
    'PTS_pg_x', 'REB_pg_x', 'AST_pg_x', 'STL_pg_x', 'BLK_pg_x', 'MIN_pg_x'
]

X = df[feature_cols]
y = df["breakout?"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model = XGBClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=4,
    random_state=42,
    eval_metric="logloss"
)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)

print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1 Score: {f1:.3f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Precision: 0.512
Recall: 0.053
F1 Score: 0.096

Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.98      0.87      1322
           1       0.51      0.05      0.10       396

    accuracy                           0.77      1718
   macro avg       0.64      0.52      0.48      1718
weighted avg       0.72      0.77      0.69      1718



In [2]:
pip install catboost

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from xgboost import XGBClassifier

df = pd.read_csv("/content/dataset.csv")

feature_cols = [
    'PTS_x', 'REB_x', 'AST_x', 'STL_x', 'BLK_x', 'MIN_x',
    'PTS_pg_x', 'REB_pg_x', 'AST_pg_x', 'STL_pg_x', 'BLK_pg_x', 'MIN_pg_x'
]

X = df[feature_cols]
y = df["breakout?"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

neg, pos = y_train.value_counts()
scale_pos_weight = neg / pos
print(f"scale_pos_weight = {scale_pos_weight:.2f}")

model = XGBClassifier(
    n_estimators=600,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    eval_metric='logloss',
    use_label_encoder=False
)

model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    verbose=100
)

y_pred = model.predict(X_test)

precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)

print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1 Score: {f1:.3f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))


scale_pos_weight = 3.34
[0]	validation_0-logloss:0.68903


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [23:16:38] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[100]	validation_0-logloss:0.61211
[200]	validation_0-logloss:0.59590
[300]	validation_0-logloss:0.59072
[400]	validation_0-logloss:0.58541
[500]	validation_0-logloss:0.58407
[599]	validation_0-logloss:0.58613
Precision: 0.337
Recall: 0.323
F1 Score: 0.330

Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.81      0.80      1322
           1       0.34      0.32      0.33       396

    accuracy                           0.70      1718
   macro avg       0.57      0.57      0.57      1718
weighted avg       0.69      0.70      0.70      1718



In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE

df = pd.read_csv("/content/dataset.csv")

feature_cols = [
    'PTS_x', 'REB_x', 'AST_x', 'STL_x', 'BLK_x', 'MIN_x',
    'PTS_pg_x', 'REB_pg_x', 'AST_pg_x', 'STL_pg_x', 'BLK_pg_x', 'MIN_pg_x'
]

X = df[feature_cols]
y = df["breakout?"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

smote = SMOTE(random_state=42, sampling_strategy='auto', k_neighbors=5)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

print(f"Before SMOTE: {y_train.value_counts().to_dict()}")
print(f"After SMOTE:  {y_train_res.value_counts().to_dict()}")

model = XGBClassifier(
    n_estimators=600,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    random_state=42,
    eval_metric='logloss',
    use_label_encoder=False
)

model.fit(
    X_train_res, y_train_res,
    eval_set=[(X_test, y_test)],
    verbose=100
)

y_pred = model.predict(X_test)

precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)

print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1 Score: {f1:.3f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Before SMOTE: {0: 5288, 1: 1581}
After SMOTE:  {0: 5288, 1: 5288}
[0]	validation_0-logloss:0.68786


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [23:16:46] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[100]	validation_0-logloss:0.62214
[200]	validation_0-logloss:0.61671
[300]	validation_0-logloss:0.61434
[400]	validation_0-logloss:0.61458
[500]	validation_0-logloss:0.61607
[599]	validation_0-logloss:0.61979
Precision: 0.323
Recall: 0.434
F1 Score: 0.370

Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.73      0.77      1322
           1       0.32      0.43      0.37       396

    accuracy                           0.66      1718
   macro avg       0.57      0.58      0.57      1718
weighted avg       0.70      0.66      0.68      1718



In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from xgboost import XGBClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from scipy.stats import uniform, randint
import numpy as np

df = pd.read_csv("/content/dataset.csv")

feature_cols = [
    'PTS_x', 'REB_x', 'AST_x', 'STL_x', 'BLK_x', 'MIN_x',
    'PTS_pg_x', 'REB_pg_x', 'AST_pg_x', 'STL_pg_x', 'BLK_pg_x', 'MIN_pg_x'
]

X = df[feature_cols]
y = df["breakout?"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

pipeline = Pipeline([
    ('smote', SMOTE(random_state=42)),
    ('xgb', XGBClassifier(
        tree_method='hist',
        predictor='gpu_predictor',
        device='cuda',
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=42
    ))
])

param_distributions = {
    'xgb__n_estimators': randint(200, 800),
    'xgb__max_depth': randint(3, 10),
    'xgb__learning_rate': uniform(0.02, 0.1),
    'xgb__subsample': uniform(0.6, 0.4),
    'xgb__colsample_bytree': uniform(0.6, 0.4),
    'xgb__gamma': uniform(0, 5),
    'xgb__reg_lambda': uniform(0.5, 2.0)
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=20,
    scoring='f1',
    cv=cv,
    verbose=2,
    n_jobs=1,
    random_state=42,
    error_score='raise'
)

try:
    search.fit(X_train, y_train)
except Exception as e:
    raise

print("\nBest params:", search.best_params_)
print(f"Best CV F1: {search.best_score_:.3f}")

best_model = search.best_estimator_
y_pred = best_model.predict(X_test)

precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)

print(f"Precision: {precision:.3f}")
print(f"Recall:    {recall:.3f}")
print(f"F1 Score:  {f1:.3f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Fitting 5 folds for each of 20 candidates, totalling 100 fits


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [23:16:54] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "predictor", "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/core.py:774: UserWarning: [23:16:54] WARNING: /workspace/src/common/error_msg.cc:62: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)


[CV] END xgb__colsample_bytree=0.749816047538945, xgb__gamma=4.75357153204958, xgb__learning_rate=0.09319939418114051, xgb__max_depth=7, xgb__n_estimators=220, xgb__reg_lambda=0.812037280884873, xgb__subsample=0.662397808134481; total time=   1.1s


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [23:16:55] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "predictor", "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[CV] END xgb__colsample_bytree=0.749816047538945, xgb__gamma=4.75357153204958, xgb__learning_rate=0.09319939418114051, xgb__max_depth=7, xgb__n_estimators=220, xgb__reg_lambda=0.812037280884873, xgb__subsample=0.662397808134481; total time=   0.8s


...

[CV] END xgb__colsample_bytree=0.7297380084021096, xgb__gamma=0.6104397735033668, xgb__learning_rate=0.055629783807697494, xgb__max_depth=8, xgb__n_estimators=424, xgb__reg_lambda=1.0442644987692706, xgb__subsample=0.8590760482165449; total time=   0.9s


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [23:17:53] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "predictor", "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[CV] END xgb__colsample_bytree=0.7297380084021096, xgb__gamma=0.6104397735033668, xgb__learning_rate=0.055629783807697494, xgb__max_depth=8, xgb__n_estimators=424, xgb__reg_lambda=1.0442644987692706, xgb__subsample=0.8590760482165449; total time=   0.9s


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [23:17:54] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "predictor", "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



Best params: {'xgb__colsample_bytree': np.float64(0.6812244898939077), 'xgb__gamma': np.float64(4.714267852789905), 'xgb__learning_rate': np.float64(0.0798865466488536), 'xgb__max_depth': 5, 'xgb__n_estimators': 285, 'xgb__reg_lambda': np.float64(2.260935678030515), 'xgb__subsample': np.float64(0.8497416192535173)}
Best CV F1: 0.405
Precision: 0.318
Recall:    0.593
F1 Score:  0.414

Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.62      0.71      1322
           1       0.32      0.59      0.41       396

    accuracy                           0.61      1718
   macro avg       0.58      0.61      0.56      1718
weighted avg       0.72      0.61      0.64      1718



In [6]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, precision_recall_curve
from xgboost import XGBClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from scipy.stats import uniform, randint
import xgboost as xgb

df = pd.read_csv("/content/dataset.csv")

feature_cols = [
    'PTS_x','REB_x','AST_x','STL_x','BLK_x','MIN_x',
    'PTS_pg_x','REB_pg_x','AST_pg_x','STL_pg_x','BLK_pg_x','MIN_pg_x'
]
X = df[feature_cols]
y = df["breakout?"].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


gpu_params = dict(tree_method="hist", predictor="gpu_predictor", device="cuda")

base_xgb = XGBClassifier(
    **gpu_params,
    eval_metric="logloss",
    random_state=42,
    use_label_encoder=False
)

pipe = Pipeline([
    ("smote", SMOTE(random_state=42)),
    ("xgb", base_xgb)
])

param_distributions = {
    "xgb__n_estimators": randint(300, 900),
    "xgb__max_depth": randint(3, 10),
    "xgb__learning_rate": uniform(0.02, 0.1),
    "xgb__subsample": uniform(0.6, 0.4),
    "xgb__colsample_bytree": uniform(0.6, 0.4),
    "xgb__gamma": uniform(0, 5),
    "xgb__min_child_weight": randint(1, 10),
    "xgb__reg_lambda": uniform(0.5, 2),
    "xgb__reg_alpha": uniform(0, 1),
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

search = RandomizedSearchCV(
    estimator=pipe,
    param_distributions=param_distributions,
    n_iter=30,
    scoring="f1",
    cv=cv,
    verbose=2,
    n_jobs=1,
    random_state=42
)

search.fit(X_train, y_train)
print("\nBest parameters:", search.best_params_)
print(f"Best CV F1: {search.best_score_:.4f}")


best_params = {k.replace("xgb__", ""): v for k, v in search.best_params_.items() if k.startswith("xgb__")}
params = {
    **best_params,
    **gpu_params,
    "eval_metric": "logloss",
    "random_state": 42
}

X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

smote = SMOTE(random_state=42)
X_tr_res, y_tr_res = smote.fit_resample(X_tr, y_tr)

dtrain = xgb.DMatrix(X_tr_res, label=y_tr_res)
dval   = xgb.DMatrix(X_val, label=y_val)

num_boost_round = int(best_params.get("n_estimators", 600))
booster = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=num_boost_round,
    evals=[(dval, "validation")],
    early_stopping_rounds=50,
    verbose_eval=False
)

final_xgb = XGBClassifier(**params)
final_xgb._Booster = booster
final_xgb.n_estimators = booster.best_iteration


y_val_prob = booster.predict(dval)
prec, rec, thresh = precision_recall_curve(y_val, y_val_prob)
f1_scores = 2 * (prec * rec) / (prec + rec + 1e-12)
best_idx = np.nanargmax(f1_scores)
best_thresh = float(thresh[max(best_idx, 0)]) if len(thresh) > 0 else 0.5
print(f"\nBest threshold for F1 (validation): {best_thresh:.4f}")
print(f"Val Precision/Recall/F1: {prec[best_idx]:.3f}/{rec[best_idx]:.3f}/{f1_scores[best_idx]:.3f}")


dtest = xgb.DMatrix(X_test)
y_test_prob = booster.predict(dtest)
y_test_pred = (y_test_prob >= best_thresh).astype(int)

precision = precision_score(y_test, y_test_pred, zero_division=0)
recall    = recall_score(y_test, y_test_pred, zero_division=0)
f1        = f1_score(y_test, y_test_pred, zero_division=0)

print(f"Precision: {precision:.3f}")
print(f"Recall:    {recall:.3f}")
print(f"F1 Score:  {f1:.3f}")
print("\nClassification Report:\n", classification_report(y_test, y_test_pred))


Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV] END xgb__colsample_bytree=0.749816047538945, xgb__gamma=4.75357153204958, xgb__learning_rate=0.09319939418114051, xgb__max_depth=7, xgb__min_child_weight=5, xgb__n_estimators=421, xgb__reg_alpha=0.15599452033620265, xgb__reg_lambda=0.6161672243363989, xgb__subsample=0.9464704583099741; total time=   0.3s
[CV] END xgb__colsample_bytree=0.749816047538945, xgb__gamma=4.75357153204958, xgb__learning_rate=0.09319939418114051, xgb__max_depth=7, xgb__min_child_weight=5, xgb__n_estimators=421, xgb__reg_alpha=0.15599452033620265, xgb__reg_lambda=0.6161672243363989, xgb__subsample=0.9464704583099741; total time=   0.3s
[CV] END xgb__colsample_bytree=0.749816047538945, xgb__gamma=4.75357153204958, xgb__learning_rate=0.09319939418114051, xgb__max_depth=7, xgb__min_child_weight=5, xgb__n_estimators=421, xgb__reg_alpha=0.15599452033620265, xgb__reg_lambda=0.6161672243363989, xgb__subsample=0.9464704583099741; total time=   0.3s
[CV] 

In [7]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import (
    precision_score, recall_score, f1_score,
    classification_report, precision_recall_curve
)
from catboost import CatBoostClassifier, Pool


df = pd.read_csv("/content/dataset.csv")

feature_cols = [
    'PTS_x','REB_x','AST_x','STL_x','BLK_x','MIN_x',
    'PTS_pg_x','REB_pg_x','AST_pg_x','STL_pg_x','BLK_pg_x','MIN_pg_x'
]
X = df[feature_cols]
y = df["breakout?"].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

neg, pos = np.bincount(y_train)
scale_pos_weight = neg / pos
print(f"Class imbalance ratio (neg/pos): {scale_pos_weight:.2f}")

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

base_cat = CatBoostClassifier(
    task_type="GPU",
    devices='0',
    eval_metric="F1",
    loss_function="Logloss",
    random_seed=42,
    verbose=False,
    class_weights=[1.0, scale_pos_weight]
)

param_grid = {
    "iterations": [500, 1000, 1500],
    "depth": [4, 5, 6, 7, 8],
    "learning_rate": np.linspace(0.01, 0.2, 10),
    "l2_leaf_reg": np.linspace(1, 10, 10),
    "border_count": [32, 64, 128],
    "bagging_temperature": np.linspace(0, 1, 5),
    "random_strength": np.linspace(0.5, 5.0, 5)
}

best_f1 = -1
best_params = None


for depth in [4, 5, 6, 7]:
    for lr in [0.02, 0.05, 0.08, 0.1]:
        model = CatBoostClassifier(
            task_type="GPU",
            devices='0',
            iterations=1000,
            depth=depth,
            learning_rate=lr,
            eval_metric="F1",
            loss_function="Logloss",
            random_seed=42,
            verbose=False,
            class_weights=[1.0, scale_pos_weight]
        )
        cv_f1 = []
        for train_idx, val_idx in cv.split(X_train, y_train):
            X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
            y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
            model.fit(X_tr, y_tr, eval_set=(X_val, y_val), early_stopping_rounds=50)
            y_val_pred = model.predict(X_val)
            f1 = f1_score(y_val, y_val_pred, zero_division=0)
            cv_f1.append(f1)
        mean_f1 = np.mean(cv_f1)
        if mean_f1 > best_f1:
            best_f1 = mean_f1
            best_params = dict(depth=depth, learning_rate=lr)
        print(f"depth={depth}, lr={lr:.3f}, mean F1={mean_f1:.4f}")

print(f"\nBest params: {best_params}, CV F1={best_f1:.4f}")


final_model = CatBoostClassifier(
    task_type="GPU",
    devices='0',
    iterations=1500,
    depth=best_params["depth"],
    learning_rate=best_params["learning_rate"],
    eval_metric="F1",
    loss_function="Logloss",
    random_seed=42,
    verbose=100,
    class_weights=[1.0, scale_pos_weight]
)

X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

train_pool = Pool(X_tr, y_tr)
val_pool = Pool(X_val, y_val)

final_model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=75, use_best_model=True, verbose=False)

y_val_prob = final_model.predict_proba(X_val)[:, 1]
prec, rec, thresh = precision_recall_curve(y_val, y_val_prob)
f1_scores = 2 * (prec * rec) / (prec + rec + 1e-12)
best_idx = np.nanargmax(f1_scores)
best_thresh = float(thresh[max(best_idx, 0)]) if len(thresh) > 0 else 0.5
print(f"\nOptimal threshold (validation F1): {best_thresh:.4f}")
print(f"Val Precision/Recall/F1: {prec[best_idx]:.3f}/{rec[best_idx]:.3f}/{f1_scores[best_idx]:.3f}")

y_test_prob = final_model.predict_proba(X_test)[:, 1]
y_test_pred = (y_test_prob >= best_thresh).astype(int)

precision = precision_score(y_test, y_test_pred, zero_division=0)
recall = recall_score(y_test, y_test_pred, zero_division=0)
f1 = f1_score(y_test, y_test_pred, zero_division=0)

print(f"Precision: {precision:.3f}")
print(f"Recall:    {recall:.3f}")
print(f"F1 Score:  {f1:.3f}")
print("\nClassification Report:\n", classification_report(y_test, y_test_pred))


Class imbalance ratio (neg/pos): 3.34
depth=4, lr=0.020, mean F1=0.4219
depth=4, lr=0.050, mean F1=0.4223
depth=4, lr=0.080, mean F1=0.4218
depth=4, lr=0.100, mean F1=0.4180
depth=5, lr=0.020, mean F1=0.4259
depth=5, lr=0.050, mean F1=0.4280
depth=5, lr=0.080, mean F1=0.4258
depth=5, lr=0.100, mean F1=0.4221
depth=6, lr=0.020, mean F1=0.4203
depth=6, lr=0.050, mean F1=0.4200
depth=6, lr=0.080, mean F1=0.4179
depth=6, lr=0.100, mean F1=0.4192
depth=7, lr=0.020, mean F1=0.4287
depth=7, lr=0.050, mean F1=0.4258
depth=7, lr=0.080, mean F1=0.4257
depth=7, lr=0.100, mean F1=0.4198

Best params: {'depth': 7, 'learning_rate': 0.02}, CV F1=0.4287

Optimal threshold (validation F1): 0.4798
Val Precision/Recall/F1: 0.314/0.775/0.447
Precision: 0.293
Recall:    0.732
F1 Score:  0.418

Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.47      0.61      1322
           1       0.29      0.73      0.42       396

    accuracy                